In [1]:
import pandas as pd
import sqlite3 as sql
import numpy as np
import os
import glob

import sqlite3
from sqlite3 import Error

# Downloading Zipped Data From Github

In [2]:
data_path = '../data/raw/owid-datasets-master/datasets/'
data_path = os.path.realpath(data_path)
folders = os.listdir(data_path)

In [3]:
os.listdir()

['.gitkeep', 'Data_Extraction.ipynb', 'OWID.db', 'Scrapping.ipynb']

Below may not work if Windows 10 Long path name not enabled. Otherwise Windows cannot open files larger than 260 charachters. One can alternatively construct the SQL Database at this step, likely more effient.

In [4]:
titles = []
dfs = []
for folder in folders[:]:
    rel_path = os.path.join(data_path, folder)
    for file in os.listdir(rel_path):
        if file.endswith('.csv'):
            final_path = os.path.join(rel_path, file)
            df = pd.read_csv(final_path)
            dfs.append(df)
            titles.append(folder)

## Using SQL

In [5]:
OWID_db = '../data/raw/owid-datasets-master/'+ "OWID.db"
conn = sqlite3.connect(OWID_db)
for title, df in zip(titles, dfs):
  df.to_sql(title, conn, index=False)
conn.close()

C:\Users\Ferna\anaconda3\lib\site-packages\pandas\core\generic.py:2602: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [6]:
conn = sqlite3.connect(OWID_db)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

In [7]:
pd.read_sql( 'SELECT * FROM "%s" '  % (tables[-1][0]), conn)

,Entity,Year,Youth mortality rates (0-14)
0,Afghanistan,1977,297.48592
1,Afghanistan,1978,289.08271
2,Afghanistan,1979,280.57551
3,Afghanistan,1980,272.02735
4,Afghanistan,1981,263.45558
...,...,...,...
8300,Zimbabwe,2013,80.98464
8301,Zimbabwe,2014,75.18350
8302,Zimbabwe,2015,70.86210
8303,Zimbabwe,2016,66.93248


# Scrapping Data from Github OWID Datasets

In [2]:
import bs4
import requests

In [3]:
url = "https://github.com/owid/owid-datasets/tree/master/datasets"

webpage = requests.get(url).text

In [4]:
soup = bs4.BeautifulSoup(webpage)

In [5]:
results = soup.find_all('a', {'class':'js-navigation-open Link--primary'})

links =[]
titles = []
for result in results:
  rel_short_link = result['href'].split('/')[-1]
  data_url = 'https://raw.githubusercontent.com/owid/owid-datasets/master/datasets/' + rel_short_link+ '/'+rel_short_link+'.csv'
  
  titles.append(result['title'])
  links.append(data_url) 
# class="js-navigation-open Link--primary"

In [6]:
df = pd.read_csv(links[-1])

In [7]:
titles[:5]

[' Literacy by years of schooling US 1947 – OECD (2014) ',
 ' Literate world population (OurWorldInData based on OECD and UNESCO)',
 '20th century deaths in US - CDC',
 'A Century of Work and Leisure - Ramey and Francis (2009)',
 'Abortion rates (per 1000 women) - Sedgh et al. (2016)']

## Using SQL


In [8]:
OWID_db = '../data/raw/owid-datasets-master/'+ "OWID.db"


In [9]:
OWID_db = '../data/raw/owid-datasets-master/'+ "OWID.db"

conn = sqlite3.connect(OWID_db)
for title, link in zip(titles, links):
  df = pd.read_csv(link)
  df.to_sql(title, conn, index=False)
conn.close()

14)
Ozone depletion impacts on skin cancer incidence - Slaper et al.
Ozone hole area and concentration - NASA
Ozone-depleting emissions index - EEA
PM2.5 concentration - State of Global Air (2017)
Parties to Montreal Protocol - UNEP
Patent and publication rates  - OWID based on World Bank & UN
Patents awarded in England, Scotland and Wales - Bottomley
Peak Farmland Projection - Ausbuel et al. (2013)
Penn World Table 9.0 - OWID's team elaboration
People experiencing homelessness in the USA (PIT) by sheltering status - HUD (2016)
Per capita energy use - World Bank (2017)
Per capita pathways for 1.5C and 2C - OWID based on CAT
Percentage deaths attributable to risk factors - IHME
Percentage gained access to improved water and sanitation 1990-2015 - WHO
Percentage of Americans living alone, by age - IPUMS
Percentage of Individuals using the Internet (ICT) - ITU (2015)
Percentage of adults living alone in the US and Canada- US Census Bureau & Statistics Canada
Percentage of persons without 

In [10]:
conn = sqlite3.connect(OWID_db)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

In [11]:
len(tables)

993

In [12]:
pd.read_sql( 'SELECT * FROM "%s" '  % (tables[-1][0]), conn)

,Entity,Year,Youth mortality rates (0-14)
0,Afghanistan,1977,297.48592
1,Afghanistan,1978,289.08271
2,Afghanistan,1979,280.57551
3,Afghanistan,1980,272.02735
4,Afghanistan,1981,263.45558
...,...,...,...
8300,Zimbabwe,2013,80.98464
8301,Zimbabwe,2014,75.18350
8302,Zimbabwe,2015,70.86210
8303,Zimbabwe,2016,66.93248
